<a href="https://colab.research.google.com/github/EhsanHonarvar/Practices/blob/main/1_Bert_StepByStep(Sentiment140).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 115.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re

In [ ]:
full_data = pd.read_csv("/content/drive/MyDrive/Thesis/training.1600000.processed.noemoticon.csv", names= ['Pos_Neg','id','date','query_string','user','Content'] , encoding = 'latin-1')
full_n = full_data.shape[0]

In [ ]:
print(full_n)

1600000


In [ ]:
print(type(full_data))
full_data.head(5)

<class 'pandas.core.frame.DataFrame'>


,Pos_Neg,id,date,query_string,user,Content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
import sklearn
from sklearn.model_selection import train_test_split

In [ ]:
print(type(full_data))

<class 'pandas.core.frame.DataFrame'>


In [ ]:
full_data.groupby(full_data.Pos_Neg).count()

,id,date,query_string,user,Content
Pos_Neg,,,,,
0,800000,800000,800000,800000,800000
4,800000,800000,800000,800000,800000


In [ ]:
train_data, dev_data = sklearn.model_selection.train_test_split(full_data, test_size = 0.04) #It was 0.04

In [ ]:
train_data.groupby(train_data.Pos_Neg).count()

,id,date,query_string,user,Content
Pos_Neg,,,,,
0,767895,767895,767895,767895,767895
4,768105,768105,768105,768105,768105


In [ ]:
train_data = train_data[:160000]
dev_data = dev_data[:64000]

In [ ]:
print(full_data.shape[0],'\n',train_data.shape[0],'\n',dev_data.shape[0])

1600000 
 1536000 
 64000


In [ ]:
# Get ground truth x and y values
train_x_raw = train_data.loc[:]["Content"]
train_y = [0.0 if y == 0 else 1.0 for y in train_data.loc[:]["Pos_Neg"]]

dev_x_raw = dev_data.loc[:]["Content"]
dev_y = [0.0 if y == 0 else 1.0 for y in dev_data.loc[:]["Pos_Neg"]]
#print (dev_y)

In [ ]:
print(train_x_raw[0:3])

873744     @itsNICKJONAS ! K I play guitar and have been ...
1145450    u r a dear thanks for the rt! follow my friend...
613557     Mostly everything shut (inc our fave beach bar...
Name: Content, dtype: object


In [ ]:
def clean(data):
  tokens = data.split()
  translation_table = str.maketrans('', '', "\"#$%&'()*+-/:;<=>@[\]^_`{|}~?!.,")
  tokens = [w.translate(translation_table) for w in tokens]
  tokens = [word.lower() for word in tokens]

  return ' '.join(tokens)

# data = []
# labels = []



In [ ]:
train_x_raw = [clean(y)  for y in train_x_raw]

In [ ]:
print(train_x_raw[0:3])

['itsnickjonas  k i play guitar and have been trying to figure out the strumming pattern to hello beautiful forever can you tell me it', 'u r a dear thanks for the rt follow my friend holdemtalkradio', 'mostly everything shut inc our fave beach bar and weather not warm sewage smell getting worse so cant use pool']


In [ ]:
print(train_y)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
from tensorflow.keras.utils import to_categorical

labels = to_categorical(train_y)

In [ ]:
data = train_x_raw

In [1]:
from transformers import BertTokenizer
import numpy as np

input_ids=[]
attention_masks=[]

bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

for sent in data:
    bert_inp = bert_tokenizer.encode_plus(sent, add_special_tokens = True, max_length =20, pad_to_max_length = True, # max_length was 64
                                        return_attention_mask = True)
    input_ids.append(bert_inp['input_ids'])
    attention_masks.append(bert_inp['attention_mask'])

input_ids = np.asarray(input_ids)
attention_masks = np.array(attention_masks)
labels = np.array(labels)


ModuleNotFoundError: ignored

In [ ]:
len(input_ids),len(attention_masks),len(labels)


(1536000, 1536000, 1536000)

In [ ]:
from sklearn.model_selection import train_test_split

train_inp, val_inp, train_label, val_label, train_mask, val_mask = train_test_split(
   input_ids, labels, attention_masks, test_size=0.20, random_state=1000)

test_inp, val_inp, test_label, val_label, test_mask, val_mask = train_test_split(
   val_inp, val_label, val_mask, test_size=0.5, random_state=1000)


In [ ]:

import tensorflow as tf
from transformers import TFBertModel, TFBertForPreTraining

SEQ_LEN = 20 # It was 64

bert = TFBertModel.from_pretrained('bert-base-uncased')

input_ids = tf.keras.layers.Input(shape=(SEQ_LEN,), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(SEQ_LEN,), name='attention_mask', dtype='int32')

embeddings = bert(input_ids, attention_mask=mask)[0]
X = tf.keras.layers.Lambda(lambda x: tf.keras.backend.mean(x, axis=1))(embeddings)
y = tf.keras.layers.Dense(2, activation='softmax', name='outputs')(X)

bert_model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5,epsilon=1e-08)

bert_model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

bert_model.summary()

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 20)]         0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 20)]         0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  109482240   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 20,                                            

In [ ]:
history = bert_model.fit([train_inp, train_mask], train_label,
                       batch_size = 32,
                       epochs = 7,
                       validation_data = ([val_inp, val_mask],val_label))

Epoch 1/7


38400/38400 [==============================] - 5504s 142ms/step - loss: 0.3926 - accuracy: 0.8208 - val_loss: 0.3728 - val_accuracy: 0.8319
Epoch 2/7
38400/38400 [==============================] - 5450s 142ms/step - loss: 0.3381 - accuracy: 0.8506 - val_loss: 0.3739 - val_accuracy: 0.8331
Epoch 3/7
38400/38400 [==============================] - 5446s 142ms/step - loss: 0.2873 - accuracy: 0.8769 - val_loss: 0.3883 - val_accuracy: 0.8311
Epoch 4/7
38400/38400 [==============================] - 5454s 142ms/step - loss: 0.2347 - accuracy: 0.9021 - val_loss: 0.4275 - val_accuracy: 0.8281
Epoch 5/7
38400/38400 [==============================] - 5448s 142ms/step - loss: 0.1885 - accuracy: 0.9235 - val_loss: 0.4918 - val_accuracy: 0.8233
Epoch 6/7
38400/38400 [==============================] - 5455s 142ms/step - loss: 0.1530 - accuracy: 0.9393 - val_loss: 0.5424 - val_accuracy: 0.8203
Epoch 7/7
38400/38400 [==============================] - 5453s 142ms/step - loss: 0.1268 - accuracy: 0.9502 - 

In [ ]:
import tensorflow as tf
from transformers import TFBertModel, TFBertForPreTraining

SEQ_LEN = 20 # It was 64

bert = TFBertModel.from_pretrained('bert-base-uncased')

input_ids = tf.keras.layers.Input(shape=(SEQ_LEN,), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(SEQ_LEN,), name='attention_mask', dtype='int32')

embeddings = bert(input_ids, attention_mask=mask)[1]
y = tf.keras.layers.Dense(2, activation='softmax', name='outputs')(embeddings)

bert_model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-08)

bert_model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

bert_model.summary()

In [ ]:
history = bert_model.fit([train_inp, train_mask], train_label,
                       batch_size = 32,
                       epochs = 5,
                       validation_data = ([val_inp, val_mask],val_label))